# E-GraphSAGE

In [1]:
from dgl import from_networkx
import sklearn
import torch.nn as nn
import torch as th
import torch.nn.functional as F
import dgl.function as fn
import networkx as nx
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import category_encoders as ce
import numpy as np
from sklearn.metrics import confusion_matrix

import os
from sklearn.utils import shuffle

from dgl.data.utils import save_graphs

#constante
size_embedding = 152
nb_batch = 5

#Data
nbclasses =  2

# Accuracy --------------------------------------------------------------------
def compute_accuracy(pred, labels):
    return (pred.argmax(1) == labels).float().mean().item()
# -----------------------------------------------------------------------------

# ------------------------------------------ Model Architecture -----------------------------------------------------------------

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
        super(SAGELayer, self).__init__()
        self.W_msg = nn.Linear(ndim_in + edims, ndim_out)
        self.W_apply = nn.Linear(ndim_in + ndim_out, ndim_out)
        self.activation = activation

    def message_func(self, edges):
        x = th.cat([edges.src['h'], edges.data['h']], 2)
        y = self.W_msg(x)
        return {'m': y}

    def forward(self, g_dgl, nfeats, efeats):
        with g_dgl.local_scope():
            g = g_dgl
            g.ndata['h'] = nfeats
            g.edata['h'] = efeats
            # Line 4 of algorithm 1 : update all because we are using a full neighborhood sampling and not a k-hop neigh sampling
            g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
            # Line 5 of algorithm 1
            g.ndata['h'] = F.relu(self.W_apply(th.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))
            return g.ndata['h']


class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation, dropout):
        super(SAGE, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(SAGELayer(ndim_in, edim, size_embedding, activation))
        self.layers.append(SAGELayer(size_embedding, edim, size_embedding, activation)) ##
        self.layers.append(SAGELayer(size_embedding, edim, ndim_out, activation))
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, g, nfeats, efeats):
        
        for i, layer in enumerate(self.layers):
            if i != 0:
                nfeats = self.dropout(nfeats)
            nfeats = layer(g, nfeats, efeats)
            # Save edge_embeddings
            # nf = 'edge_embeddings'+str(i)+'.txt'
            # sourceFile = open(nf, 'w')
            # print(nfeats, file = sourceFile)
        return nfeats.sum(1)
        # Return a list of node features [[node1_feature1, node1_feature2, ...], [node2_feature1, node2_feature2, ...], ...]
    
class MLPPredictor(nn.Module):
    def __init__(self, in_features, out_classes):
        super().__init__()
        self.W = nn.Linear(in_features * 2, out_classes)

    def apply_edges(self, edges):
        h_u = edges.src['h']
        h_v = edges.dst['h']
        v = th.cat([h_u, h_v], 1)
        # if(pr == True):
            # sourceFile = open(filename, 'w')
            # if pr:
                # print(v, file = sourceFile)
            # sourceFile.close()
        score = self.W(v)
        return {'score': score}

    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata['h'] = h
            # Update the features of the specified edges by the provided function
            # DGLGraph.apply_edges(func, edges='__ALL__', etype=None, inplace=False)
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']

class Model(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation, dropout):
        super().__init__()
        self.gnn = SAGE(ndim_in, ndim_out, edim, activation, dropout)
        self.pred = MLPPredictor(ndim_out, nbclasses)
    def forward(self, g, nfeats, efeats, eweight = None):
        if eweight != None:
            # apply eweight on the graph
            efe = []
            for i, x in enumerate(eweight):
                efe.append(list(th.Tensor.cpu(g.edata['h'][i][0]).detach().numpy() * th.Tensor.cpu(x).detach().numpy()))

            efe = th.FloatTensor(efe).cuda()
            efe = th.reshape(efe, (efe.shape[0], 1, efe.shape[1]))
            g.edata['h'] = efe = efe

        h = self.gnn(g, nfeats, efeats)
        # h = list of node features [[node1_feature1, node1_feature2, ...], [node2_feature1, node2_feature2, ...], ...]
        return self.pred(g, h)

# -------------------------------------------------------------------------------------------------------------------------------

## Graph + Ablation

In [3]:
# # --------------------------------------------------- MAIN -----------------------------------------------------------

import copy


# Model *******************************************************************************************
# G1.ndata['h'].shape[2] = sizeh = 76 dans ANIDS
# model1 = Model(G1.ndata['h'].shape[2], size_embedding, G1.ndata['h'].shape[2], F.relu, 0.2).cuda()
model1 = Model(76, size_embedding, 76, F.relu, 0.2).cuda()
opt = th.optim.Adam(model1.parameters())

model1_ab = Model(76, size_embedding, 76, F.relu, 0.2).cuda()
opt_ab = th.optim.Adam(model1_ab.parameters())


# Classes
clss = ['BENIGN', 'Brute Force', 'XSS', 'Sql Injection', 'Heartbleed', 'DoS Hulk', 'DDoS', 'PortScan', 'FTP-Patator', 'Bot', 'DoS slowloris', 'DoS GoldenEye', 'DoS Slowhttptest', 'SSH-Patator', 'Infiltration']
# Classes mpping
clss_mpping = {}
cpt = 0
for x in clss:
    clss_mpping[x] = cpt
    cpt += 1
print(clss_mpping)



path, dirs, files = next(os.walk("/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/input/Dataset/GlobalDataset/Splitted/"))
file_count = len(files)


for nb_files in range(file_count):
    data1 = pd.read_csv(f'{path}{files[nb_files]}', encoding="ISO-8859–1", dtype = str)

    print(f'{files[nb_files]} ++++++++++++++++++++++++++++++++++++++++++++++')
    print("nb total instances in the file : ", len(data1.values))

    print("++++++++++++++++++++++++++++ Train ++++++++++++++++++++++++++++++++")
    
    # Delete two columns (U and V in the excel)
    cols = list(set(list(data1.columns )) - set(list(['Flow Bytes/s',' Flow Packets/s'])) )
    data1 = data1[cols]

    # Mise en forme des noeuds
    data1[' Source IP'] = data1[' Source IP'].apply(str)
    data1[' Source Port'] = data1[' Source Port'].apply(str)
    data1[' Destination IP'] = data1[' Destination IP'].apply(str)
    data1[' Destination Port'] = data1[' Destination Port'].apply(str)
    data1[' Source IP'] = data1[' Source IP'] + ':' + data1[' Source Port']
    data1[' Destination IP'] = data1[' Destination IP'] + ':' + data1[' Destination Port']

    data1.drop(columns=['Flow ID',' Source Port',' Destination Port',' Timestamp'], inplace=True)
    
    data1['DetailedLabel'] = data1[' Label']
    data1['DetailedLabel'] = data1['DetailedLabel'].apply(str)
    # Classes mpping for the DetailedLabel
    data1 = data1.replace({'DetailedLabel': clss_mpping})

    # -------------------- ????????????????????????????????????????? --------------------
    # simply do : nom = list(data1[' Label'].unique())
    nom = []
    nom = nom + [data1[' Label'].unique()[0]]
    for i in range(1, len(data1[' Label'].unique())):
        nom = nom + [data1[' Label'].unique()[i]]
    
    nom.insert(0, nom.pop(nom.index('BENIGN')))

    # Naming the two classes BENIGN {0} / Any Intrusion {1}
    data1[' Label'].replace(nom[0], 0,inplace = True)
    for i in range(1,len(data1[' Label'].unique())):
        data1[' Label'].replace(nom[i], 1,inplace = True)
    
    ##################### LABELS FREQ #######################################
    print()
    print("labels freq after changing labels to binary")
    counts = list(data1[' Label'].value_counts().to_dict().items())
    for j, x in enumerate(counts):
        x = list(x)
        x[1] = x[1] / len(data1)
        counts[j] = x
    print({f'{files[nb_files]}' : counts})
    ##############################################################################

    data1.rename(columns={" Label": "label"},inplace = True)
    label1 = data1.label
    data1.drop(columns=['label'],inplace = True)

    # ******** At this step data1 contains only the data without label column
    # ******** The label column is stored in the label variale 

    # split train and test
    data1 =  pd.concat([data1, label1], axis=1) # ??????? WHY ?
        

    # Split
    X1_train, X1_test, y1_train, y1_test = train_test_split(data1, label1, test_size=0.3, random_state=123, stratify= label1)

    
    # Create mini batches on the Train set
    X1_train = shuffle(X1_train)
    a = b = mean_macro_f1 = 0
    for batch in range(1, nb_batch + 1):
        print(f"+++++++++++++++++ Batch {batch} ++++++++++++++++")
        a = b
        b = int(len(X1_train) / nb_batch) * batch
        if batch == nb_batch :
            b = len(X1_train)
        # The batch :
        X1_train_batched = X1_train.iloc[a:b]
        # y1_train_batched = y1_train.iloc[a:b]
        y1_train_batched = X1_train_batched['label']        
        
        print("nb Train instances : ", len(X1_train_batched.values))

        # for non numerical attributes (categorical data)
        # Since we have a binary classification, the category values willl be replaced with the posterior probability (p(target = Ti | category = Cj))
        # TargetEncoding is also called MeanEncoding, cuz it simply replace each value with (target_i_count_on_category_j) / (total_occurences_of_category_j)
        encoder1 = ce.TargetEncoder(cols=[' Protocol',  'Fwd PSH Flags', ' Fwd URG Flags', ' Bwd PSH Flags', ' Bwd URG Flags'])
        encoder1.fit(X1_train_batched, y1_train_batched)
        X1_train_batched = encoder1.transform(X1_train_batched)

        # scaler (normalization)
        scaler1 = StandardScaler()

        # Manipulate flow content (all columns except : label, Source IP & Destination IP)
        cols_to_norm1 = list(set(list(X1_train_batched.iloc[:, :].columns )) - set(list(['label', ' Source IP', ' Destination IP', 'DetailedLabel'])) )
        X1_train_batched[cols_to_norm1] = scaler1.fit_transform(X1_train_batched[cols_to_norm1])
        
        print("cols_to_norm1 : ", cols_to_norm1)
        print("cols_to_norm1[0] : ", cols_to_norm1[0])
        print("X1_train_batched[cols_to_norm1[0]] : ", X1_train_batched[cols_to_norm1[0]])

        ## Create the h attribute that will contain the content of our flows
        X1_train_batched['h'] = X1_train_batched[ cols_to_norm1 ].values.tolist()
        print("X1_train_batched['h'] : ", X1_train_batched['h'])
        # size of the list containig the content of our flows
        sizeh = len(cols_to_norm1)


        # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        # Before training the data :
        # We need to delete all the attributes (cols_to_norm1) to have the {Source IP, Destination IP, label, h} representation
        X1_train_batched.drop(columns = cols_to_norm1, inplace = True)
        
        # Edge index
        X1_train_batched['Edge_indx'] = list(range(len(X1_train_batched.values)))

        # Then we need to Swap {label, h} Columns to have the {Source IP, Destination IP, h, label} representation
        columns_titles = [' Source IP', ' Destination IP', 'h', 'label', 'Edge_indx', 'DetailedLabel']
        X1_train_batched = X1_train_batched.reindex(columns=columns_titles)
        # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

        # ------------------------------------------- Creating the Graph Representation -------------------------------------------------------------
        # Create our Multigraph
        G1 = nx.from_pandas_edgelist(X1_train_batched, " Source IP", " Destination IP", ['h','label', 'Edge_indx', 'DetailedLabel'], create_using=nx.MultiDiGraph())
        print("initial nx multigraph G1 : ", G1)

        G1 = from_networkx(G1, edge_attrs=['h','label', 'Edge_indx', 'DetailedLabel'] )
        print("G1.edata['h'] after converting it to a dgl graph : ", len(G1.edata['h']))

        # nodes data // G1.edata['h'].shape[1] : sizeh = number of attributes in a flow
        G1.ndata['h'] = th.ones(G1.num_nodes(), G1.edata['h'].shape[1])
        # edges data // we create a tensor bool array that will represent the train mask
        G1.edata['train_mask'] = th.ones(len(G1.edata['h']), dtype=th.bool)

        # Reshape both tensor lists to a single value in each element for both axis
        G1.ndata['h'] = th.reshape(G1.ndata['h'], (G1.ndata['h'].shape[0], 1, G1.ndata['h'].shape[1]))
        G1.edata['h'] = th.reshape(G1.edata['h'], (G1.edata['h'].shape[0], 1, G1.edata['h'].shape[1]))
        print("G1.edata['h'] after reshape : ", len(G1.edata['h']))
        # ------------------------------------------- --------------------------------- -------------------------------------------------------------

        

        
        # Ablation *************************************************************************
        X1_train_ab = copy.deepcopy(X1_train_batched)

        # print("data IP Addr before changing them : ")
        # print(X1_train_ab[[' Source IP', ' Destination IP']])

        X1_train_ab = X1_train_ab.drop(' Source IP', axis=1)
        X1_train_ab = X1_train_ab.drop(' Destination IP', axis=1)
        
        X1_train_ab[' Source IP'] = list(range(len(X1_train_ab.values)))
        X1_train_ab[' Destination IP'] = list(range(len(X1_train_ab.values), 2 * len(X1_train_ab.values)))

        print()
        # print("data IP Addr after changing them : ")
        # print(X1_train_ab[[' Source IP', ' Destination IP']])
        
        # ------------------------------------------- Creating the Graph Representation -------------------------------------------------------------
        # Create our Ablation Multigraph
        G1_ab = nx.from_pandas_edgelist(X1_train_ab, " Source IP", " Destination IP", ['h','label', 'Edge_indx', 'DetailedLabel'], create_using=nx.MultiDiGraph())
        print("initial nx multigraph G1_ab : ", G1_ab)

        G1_ab = from_networkx(G1_ab, edge_attrs=['h','label', 'Edge_indx', 'DetailedLabel'] )
        print("G1_ab.edata['h'] after converting it to a dgl graph : ", len(G1_ab.edata['h']))

        # nodes data // G1_ab.edata['h'].shape[1] : sizeh = number of attributes in a flow
        G1_ab.ndata['h'] = th.ones(G1_ab.num_nodes(), G1_ab.edata['h'].shape[1])
        # edges data // we create a tensor bool array that will represent the train mask
        G1_ab.edata['train_mask'] = th.ones(len(G1_ab.edata['h']), dtype=th.bool)

        # Reshape both tensor lists to a single value in each element for both axis
        G1_ab.ndata['h'] = th.reshape(G1_ab.ndata['h'], (G1_ab.ndata['h'].shape[0], 1, G1_ab.ndata['h'].shape[1]))
        G1_ab.edata['h'] = th.reshape(G1_ab.edata['h'], (G1_ab.edata['h'].shape[0], 1, G1_ab.edata['h'].shape[1]))
        print("G1_ab.edata['h'] after reshape : ", len(G1_ab.edata['h']))
        # ------------------------------------------- --------------------------------- -------------------------------------------------------------
        
        # ***********************************************************************************
        
        
        # ------------------------------------------- Model -----------------------------------------------------------------------------------------
        ## use of model
        from sklearn.utils import class_weight
        class_weights1 = class_weight.compute_class_weight(class_weight = 'balanced',
                                                        classes = np.unique(G1.edata['label'].cpu().numpy()),
                                                        y = G1.edata['label'].cpu().numpy())
        class_weights1 = th.FloatTensor(class_weights1).cuda()
        criterion1 = nn.CrossEntropyLoss(weight = class_weights1)
        G1 = G1.to('cuda:0')

        node_features1 = G1.ndata['h']
        edge_features1 = G1.edata['h']

        edge_label1 = G1.edata['label']
        train_mask1 = G1.edata['train_mask']


        # to print
        pr = True
        # True if you want to print the embedding vectors
        # the name of the file where the vectors are printed
        filename = './models/M1_weights.txt'
        
        print("\\\\\\\\\\\\\\\\ NORMAL \\\\\\\\\\\\\\\\\\")

        for epoch in range(1, 1000):
            pred = model1(G1, node_features1, edge_features1).cuda()
            loss = criterion1(pred[train_mask1], edge_label1[train_mask1])
            opt.zero_grad()
            loss.backward()
            opt.step()
            if epoch % 100 == 0:
                print('Training acc:', compute_accuracy(pred[train_mask1], edge_label1[train_mask1]), loss)

        pred1 = model1(G1, node_features1, edge_features1).cuda()
        pred1 = pred1.argmax(1)
        pred1 = th.Tensor.cpu(pred1).detach().numpy()
        edge_label1 = th.Tensor.cpu(edge_label1).detach().numpy()

        print('Train metrics :')
        print("Accuracy : ", sklearn.metrics.accuracy_score(edge_label1, pred1))
        print("Precision : ", sklearn.metrics.precision_score(edge_label1, pred1, labels = [0,1]))
        print("Recall : ", sklearn.metrics.recall_score(edge_label1, pred1, labels = [0,1]))
        print("f1_score : ", sklearn.metrics.f1_score(edge_label1, pred1, labels=[0,1]))
        
        

        
        # ------------------------------------------- AB Model -----------------------------------------------------------------------------------------
        ## use of model
        from sklearn.utils import class_weight
        class_weights1_ab = class_weight.compute_class_weight(class_weight = 'balanced',
                                                        classes = np.unique(G1_ab.edata['label'].cpu().numpy()),
                                                        y = G1_ab.edata['label'].cpu().numpy())
        class_weights1_ab = th.FloatTensor(class_weights1_ab).cuda()
        criterion1_ab = nn.CrossEntropyLoss(weight = class_weights1_ab)
        G1_ab = G1_ab.to('cuda:0')

        node_features1_ab = G1_ab.ndata['h']
        edge_features1_ab = G1_ab.edata['h']

        edge_label1_ab = G1_ab.edata['label']
        train_mask1_ab = G1_ab.edata['train_mask']
        
        print("\\\\\\\\\\\\\\\\ ABLATION \\\\\\\\\\\\\\\\\\")
        
        for epoch in range(1, 1000):
            pred_ab = model1_ab(G1_ab, node_features1_ab, edge_features1_ab).cuda()
            loss_ab = criterion1_ab(pred_ab[train_mask1], edge_label1_ab[train_mask1])
            opt_ab.zero_grad()
            loss_ab.backward()
            opt_ab.step()
            if epoch % 100 == 0:
                print('Training acc:', compute_accuracy(pred_ab[train_mask1], edge_label1_ab[train_mask1]), loss_ab)

        pred1_ab = model1_ab(G1_ab, node_features1_ab, edge_features1_ab).cuda()
        pred1_ab = pred1_ab.argmax(1)
        pred1_ab = th.Tensor.cpu(pred1_ab).detach().numpy()
        edge_label1_ab = th.Tensor.cpu(edge_label1_ab).detach().numpy()

        print('Train metrics :')
        print("Accuracy : ", sklearn.metrics.accuracy_score(edge_label1_ab, pred1_ab))
        print("Precision : ", sklearn.metrics.precision_score(edge_label1_ab, pred1_ab, labels = [0,1]))
        print("Recall : ", sklearn.metrics.recall_score(edge_label1_ab, pred1_ab, labels = [0,1]))
        print("f1_score : ", sklearn.metrics.f1_score(edge_label1_ab, pred1_ab, labels=[0,1]))

        



    # ------------------------------------------------ Test ---------------------------------------------------------------------
    print("++++++++++++++++++++++++++++ Test ++++++++++++++++++++++++++++++++")
    print("nb Test instances : ", len(X1_test.values))
    X1_test = encoder1.transform(X1_test)
    X1_test[cols_to_norm1] = scaler1.transform(X1_test[cols_to_norm1])

    # Save X1_test for XAI
    X1_test.to_csv(f'/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/input/Dataset/XAI/X_test{nb_files}.csv', sep=',', index = False)

    X1_test['h'] = X1_test[ cols_to_norm1 ].values.tolist()

    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # Before training the data :
    # We need to delete all the attributes (cols_to_norm1) to have the {Source IP, Destination IP, label, h} representation
    X1_test.drop(columns = cols_to_norm1, inplace = True)
    
    # Edge index
    X1_test['Edge_indx'] = list(range(len(X1_test.values)))

    # Then we need to Swap {label, h} Columns to have the {Source IP, Destination IP, h, label} representation
    columns_titles = [' Source IP', ' Destination IP', 'h', 'label', 'Edge_indx', 'DetailedLabel']
    X1_test = X1_test.reindex(columns=columns_titles)
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    
    # Graph Construction
    G1_test = nx.from_pandas_edgelist(X1_test, " Source IP", " Destination IP", ['h','label', 'Edge_indx', 'DetailedLabel'],create_using=nx.MultiDiGraph())
    # G1_test = G1_test.to_directed()
    G1_test = from_networkx(G1_test,edge_attrs=['h','label', 'Edge_indx', 'DetailedLabel'] )
    # actual1 = G1_test.edata.pop('label')
    actual1 = G1_test.edata['label']
    G1_test.ndata['feature'] = th.ones(G1_test.num_nodes(), G1.ndata['h'].shape[2])
    G1_test.ndata['feature'] = th.reshape(G1_test.ndata['feature'], (G1_test.ndata['feature'].shape[0], 1, G1_test.ndata['feature'].shape[1]))
    G1_test.edata['h'] = th.reshape(G1_test.edata['h'], (G1_test.edata['h'].shape[0], 1, G1_test.edata['h'].shape[1]))
    G1_test = G1_test.to('cuda:0')
    node_features_test1 = G1_test.ndata['feature']
    edge_features_test1 = G1_test.edata['h']
    
    
    
    # Ablation *************************************************************************
    X1_test_ab = copy.deepcopy(X1_test)

    # print("data IP Addr before changing them : ")
    # print(X1_test_ab[[' Source IP', ' Destination IP']])

    X1_test_ab = X1_test_ab.drop(' Source IP', axis=1)
    X1_test_ab = X1_test_ab.drop(' Destination IP', axis=1)

    X1_test_ab[' Source IP'] = list(range(len(X1_test_ab.values)))
    X1_test_ab[' Destination IP'] = list(range(len(X1_test_ab.values), 2 * len(X1_test_ab.values)))

    print()
    # print("data IP Addr after changing them : ")
    # print(X1_test_ab[[' Source IP', ' Destination IP']])
    
    # Graph Construction
    G1_test_ab = nx.from_pandas_edgelist(X1_test_ab, " Source IP", " Destination IP", ['h','label', 'Edge_indx', 'DetailedLabel'],create_using=nx.MultiDiGraph())
    G1_test_ab = from_networkx(G1_test_ab, edge_attrs=['h','label', 'Edge_indx', 'DetailedLabel'] )
    actual1_ab = G1_test_ab.edata['label']
    G1_test_ab.ndata['feature'] = th.ones(G1_test_ab.num_nodes(), G1_ab.ndata['h'].shape[2])
    G1_test_ab.ndata['feature'] = th.reshape(G1_test_ab.ndata['feature'], (G1_test_ab.ndata['feature'].shape[0], 1, G1_test_ab.ndata['feature'].shape[1]))
    G1_test_ab.edata['h'] = th.reshape(G1_test_ab.edata['h'], (G1_test_ab.edata['h'].shape[0], 1, G1_test_ab.edata['h'].shape[1]))
    G1_test_ab = G1_test_ab.to('cuda:0')
    node_features_test1_ab = G1_test_ab.ndata['feature']
    edge_features_test1_ab = G1_test_ab.edata['h']
    
    # ***********************************************************************************

    print("\\\\\\\\\\\\\\\\ NORMAL \\\\\\\\\\\\\\\\\\")
    print("nb instances : ", len(X1_test.values))

    test_pred1 = model1(G1_test, node_features_test1, edge_features_test1).cuda()
    test_pred1 = test_pred1.argmax(1)
    test_pred1 = th.Tensor.cpu(test_pred1).detach().numpy()

    print('Metrics : ')
    print("Accuracy : ", sklearn.metrics.accuracy_score(actual1, test_pred1))
    print("Precision : ", sklearn.metrics.precision_score(actual1, test_pred1, labels = [0,1]))
    print("Recall : ", sklearn.metrics.recall_score(actual1, test_pred1, labels = [0,1]))
    print("f1_score : ", sklearn.metrics.f1_score(actual1, test_pred1, labels = [0,1]))
    
    
    print("\\\\\\\\\\\\\\\\ ABLATION \\\\\\\\\\\\\\\\\\")
    print("nb instances : ", len(X1_test_ab.values))

    test_pred1_ab = model1_ab(G1_test_ab, node_features_test1_ab, edge_features_test1_ab).cuda()
    test_pred1_ab = test_pred1_ab.argmax(1)
    test_pred1_ab = th.Tensor.cpu(test_pred1_ab).detach().numpy()

    print('Metrics : ')
    print("Accuracy : ", sklearn.metrics.accuracy_score(actual1_ab, test_pred1_ab))
    print("Precision : ", sklearn.metrics.precision_score(actual1_ab, test_pred1_ab, labels = [0,1]))
    print("Recall : ", sklearn.metrics.recall_score(actual1_ab, test_pred1_ab, labels = [0,1]))
    print("f1_score : ", sklearn.metrics.f1_score(actual1_ab, test_pred1_ab, labels = [0,1]))

    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

{'BENIGN': 0, 'Brute Force': 1, 'XSS': 2, 'Sql Injection': 3, 'Heartbleed': 4, 'DoS Hulk': 5, 'DDoS': 6, 'PortScan': 7, 'FTP-Patator': 8, 'Bot': 9, 'DoS slowloris': 10, 'DoS GoldenEye': 11, 'DoS Slowhttptest': 12, 'SSH-Patator': 13, 'Infiltration': 14}
CIC-IDS-2017-Dataset4.csv ++++++++++++++++++++++++++++++++++++++++++++++
nb total instances in the file :  460167
++++++++++++++++++++++++++++ Train ++++++++++++++++++++++++++++++++

labels freq after changing labels to binary
{'CIC-IDS-2017-Dataset4.csv': [[0, 0.7582073464633492], [1, 0.24179265353665083]]}
+++++++++++++++++ Batch 1 ++++++++++++++++
nb Train instances :  64423
cols_to_norm1 :  [' Total Backward Packets', ' Idle Min', ' Subflow Bwd Bytes', ' Fwd Packet Length Min', ' Subflow Bwd Packets', ' Avg Fwd Segment Size', ' Fwd IAT Max', 'Fwd IAT Total', ' Idle Max', 'Total Length of Fwd Packets', 'Bwd Packet Length Max', ' Bwd PSH Flags', 'Bwd Avg Bulk Rate', ' Packet Length Mean', 'Bwd IAT Total', ' Flow IAT Mean', ' Avg Bwd Se

initial nx multigraph G1 :  MultiDiGraph with 58925 nodes and 64423 edges
G1.edata['h'] after converting it to a dgl graph :  64423
G1.edata['h'] after reshape :  64423

initial nx multigraph G1_ab :  MultiDiGraph with 128846 nodes and 64423 edges
G1_ab.edata['h'] after converting it to a dgl graph :  64423
G1_ab.edata['h'] after reshape :  64423
\\\\\\\\ NORMAL \\\\\\\\\
Training acc: 0.9898793697357178 tensor(0.0198, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9900345802307129 tensor(0.0196, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9906244277954102 tensor(0.0188, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9908262491226196 tensor(0.0190, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9910125136375427 tensor(0.0189, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9905623197555542 tensor(0.0192, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9914005398750305 tensor(0.0189, device='cuda:0', gr

X1_train_batched['h'] :  370142    [-0.03657232891882459, -0.3563427367803997, -0...
87647     [-0.029987846108918553, -0.3563427367803997, -...
217529    [-0.04315681172873062, -0.3563427367803997, -0...
459083    [0.0029345679406116137, -0.3563427367803997, -...
5284      [-0.029987846108918553, -0.3563427367803997, -...
                                ...                        
160844    [-0.03657232891882459, -0.3563427367803997, -0...
381668    [-0.03657232891882459, -0.3563427367803997, -0...
18955     [-0.010234397679200454, -0.11541276042716997, ...
179506    [-0.03657232891882459, -0.3563427367803997, -0...
384223    [-0.00364991486929442, 3.028349112102859, 0.00...
Name: h, Length: 64423, dtype: object
initial nx multigraph G1 :  MultiDiGraph with 58896 nodes and 64423 edges
G1.edata['h'] after converting it to a dgl graph :  64423
G1.edata['h'] after reshape :  64423

initial nx multigraph G1_ab :  MultiDiGraph with 128846 nodes and 64423 edges
G1_ab.edata['h'] after conver

nb Test instances :  138051

\\\\\\\\ NORMAL \\\\\\\\\
nb instances :  138051
Metrics : 
Accuracy :  0.9917204511376231
Precision :  0.9773164736888863
Recall :  0.9887058118633912
f1_score :  0.9829781530625922
\\\\\\\\ ABLATION \\\\\\\\\
nb instances :  138051
Metrics : 
Accuracy :  0.9671932836415528
Precision :  0.888921840877841
Recall :  0.9877471539844218
f1_score :  0.9357324289424018
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
CIC-IDS-2017-Dataset1.csv ++++++++++++++++++++++++++++++++++++++++++++++
nb total instances in the file :  460165
++++++++++++++++++++++++++++ Train ++++++++++++++++++++++++++++++++

labels freq after changing labels to binary
{'CIC-IDS-2017-Dataset1.csv': [[0, 0.7577868807927591], [1, 0.24221311920724087]]}
+++++++++++++++++ Batch 1 ++++++++++++++++
nb Train instances :  64423
cols_to_norm1 :  [' Total Backward Packets', ' Idle Min', ' Subflow Bwd Bytes', ' Fwd Packet Length Min', ' Subflow Bwd Packets', ' Avg Fwd Segment Size', ' Fwd I

initial nx multigraph G1 :  MultiDiGraph with 58961 nodes and 64423 edges
G1.edata['h'] after converting it to a dgl graph :  64423
G1.edata['h'] after reshape :  64423

initial nx multigraph G1_ab :  MultiDiGraph with 128846 nodes and 64423 edges
G1_ab.edata['h'] after converting it to a dgl graph :  64423
G1_ab.edata['h'] after reshape :  64423
\\\\\\\\ NORMAL \\\\\\\\\
Training acc: 0.991509199142456 tensor(0.0176, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9919127821922302 tensor(0.0169, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9918817281723022 tensor(0.0172, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9922077059745789 tensor(0.0171, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9921456575393677 tensor(0.0169, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9921456575393677 tensor(0.0166, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9911832213401794 tensor(0.0171, device='cuda:0', gra

...

X1_train_batched['h'] :  9506      [-0.011971442923664196, -0.35423631434153063, ...
114260    [0.0026034909086661544, -0.35423631434153063, ...
7722      [-0.004683976007499021, 0.05876350311079469, -...
163070    [-0.026546376755994548, -0.35423631434153063, ...
390407    [-0.01925890983982937, -0.35423631434153063, -...
                                ...                        
106510    [-0.026546376755994548, -0.35423631434153063, ...
429722    [-0.01925890983982937, -0.35423631434153063, -...
384059    [-0.008327709465581609, -0.35423631434153063, ...
277526    [-0.01925890983982937, -0.35423631434153063, -...
215026    [-0.0010402425494164334, -0.35423631434153063,...
Name: h, Length: 64423, dtype: object
initial nx multigraph G1 :  MultiDiGraph with 58916 nodes and 64423 edges
G1.edata['h'] after converting it to a dgl graph :  64423
G1.edata['h'] after reshape :  64423

initial nx multigraph G1_ab :  MultiDiGraph with 128846 nodes and 64423 edges
G1_ab.edata['h'] after conver

nb Test instances :  138051

\\\\\\\\ NORMAL \\\\\\\\\
nb instances :  138051
Metrics : 
Accuracy :  0.9910395433571652
Precision :  0.9674180387163981
Recall :  0.9966909134271404
f1_score :  0.9818363361379087
\\\\\\\\ ABLATION \\\\\\\\\
nb instances :  138051
Metrics : 
Accuracy :  0.9829628180889671
Precision :  0.942515038020656
Recall :  0.9902814214166468
f1_score :  0.9658079897656568
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
CIC-IDS-2017-Dataset3.csv ++++++++++++++++++++++++++++++++++++++++++++++
nb total instances in the file :  460164
++++++++++++++++++++++++++++ Train ++++++++++++++++++++++++++++++++

labels freq after changing labels to binary
{'CIC-IDS-2017-Dataset3.csv': [[0, 0.757532097252284], [1, 0.24246790274771604]]}
+++++++++++++++++ Batch 1 ++++++++++++++++
nb Train instances :  64422
cols_to_norm1 :  [' Total Backward Packets', ' Idle Min', ' Subflow Bwd Bytes', ' Fwd Packet Length Min', ' Subflow Bwd Packets', ' Avg Fwd Segment Size', ' Fwd IA

initial nx multigraph G1 :  MultiDiGraph with 59083 nodes and 64422 edges
G1.edata['h'] after converting it to a dgl graph :  64422
G1.edata['h'] after reshape :  64422

initial nx multigraph G1_ab :  MultiDiGraph with 128844 nodes and 64422 edges
G1_ab.edata['h'] after converting it to a dgl graph :  64422
G1_ab.edata['h'] after reshape :  64422
\\\\\\\\ NORMAL \\\\\\\\\
Training acc: 0.9897861480712891 tensor(0.0199, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9899103045463562 tensor(0.0189, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9901276230812073 tensor(0.0192, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9907019734382629 tensor(0.0187, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9900189638137817 tensor(0.0183, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9900965690612793 tensor(0.0184, device='cuda:0', grad_fn=<NllLossBackward0>)
Training acc: 0.9903294444084167 tensor(0.0182, device='cuda:0', gr

X1_train_batched['h'] :  215916    [-0.029445652666745474, -0.3578530493299633, -...
279183    [-0.0057702057152199835, 3.0525457885894047, 0...
416113    [-0.029445652666745474, -0.3578530493299633, -...
129087    [-0.041283376142508224, 0.4997560591479976, -0...
20894     [-0.029445652666745474, -0.3578530493299633, -...
                                ...                        
431625    [-0.029445652666745474, -0.3578530493299633, -...
289322    [-0.035364514404626846, -0.3578530493299633, -...
447925    [-0.029445652666745474, -0.3578530493299633, -...
117087    [-0.029445652666745474, -0.3578530493299633, -...
52662     [0.00014865602266138936, 0.8444027102185799, -...
Name: h, Length: 64422, dtype: object
initial nx multigraph G1 :  MultiDiGraph with 58861 nodes and 64422 edges
G1.edata['h'] after converting it to a dgl graph :  64422
G1.edata['h'] after reshape :  64422

initial nx multigraph G1_ab :  MultiDiGraph with 128844 nodes and 64422 edges
G1_ab.edata['h'] after conver

nb Test instances :  138050

\\\\\\\\ NORMAL \\\\\\\\\
nb instances :  138050
Metrics : 
Accuracy :  0.9908294096341905
Precision :  0.9682475066151028
Recall :  0.9948017805395394
f1_score :  0.981345043027231
\\\\\\\\ ABLATION \\\\\\\\\
nb instances :  138050
Metrics : 
Accuracy :  0.9835132198478812
Precision :  0.9456841624046401
Recall :  0.9887969408179726
f1_score :  0.9667601355298515
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


### ACC + F1

In [6]:
print("\\\\\\\\\\\\\\\\ NORMAL \\\\\\\\\\\\\\\\\\")
print("nb instances : ", len(X1_test.values))

test_pred1 = model1(G1_test, node_features_test1, edge_features_test1).cuda()
test_pred1 = test_pred1.argmax(1)
test_pred1 = th.Tensor.cpu(test_pred1).detach().numpy()

print('Metrics : ')
print("Accuracy : ", sklearn.metrics.accuracy_score(actual1, test_pred1))
print("Precision : ", sklearn.metrics.precision_score(actual1, test_pred1, labels = [0,1]))
print("Recall : ", sklearn.metrics.recall_score(actual1, test_pred1, labels = [0,1]))
print("f1_score : ", sklearn.metrics.f1_score(actual1, test_pred1, labels = [0,1]))


print("\\\\\\\\\\\\\\\\ ABLATION \\\\\\\\\\\\\\\\\\")
print("nb instances : ", len(X1_test_ab.values))

test_pred1_ab = model1_ab(G1_test_ab, node_features_test1_ab, edge_features_test1_ab).cuda()
test_pred1_ab = test_pred1_ab.argmax(1)
test_pred1_ab = th.Tensor.cpu(test_pred1_ab).detach().numpy()

print('Metrics : ')
print("Accuracy : ", sklearn.metrics.accuracy_score(actual1_ab, test_pred1_ab))
print("Precision : ", sklearn.metrics.precision_score(actual1_ab, test_pred1_ab, labels = [0,1]))
print("Recall : ", sklearn.metrics.recall_score(actual1_ab, test_pred1_ab, labels = [0,1]))
print("f1_score : ", sklearn.metrics.f1_score(actual1_ab, test_pred1_ab, labels = [0,1]))

\\\\\\\\ NORMAL \\\\\\\\\
nb instances :  138050
Metrics : 
Accuracy :  0.9910032596885187
Precision :  0.968351303438054
Recall :  0.9954291518537328
f1_score :  0.9817035443858461
\\\\\\\\ ABLATION \\\\\\\\\
nb instances :  138050
Metrics : 
Accuracy :  0.9836001448750453
Precision :  0.9458301191394531
Recall :  0.9890060645893706
f1_score :  0.9669363554049712


### Save X_Test + Models + Graphs + Predictions

In [7]:
X1_test.to_csv(f'/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/DetailedLabel/Pandas_X_Test_DetailedLabel.csv', sep=',', index = False)

# Models
th.save(model1.state_dict(), '/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/DetailedLabel/Models/Pandas_GNN_DetailedLabel.pt')
th.save(model1_ab.state_dict(), '/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/DetailedLabel/Models/Pandas_GNN_ab_DetailedLabel.pt')

# Graphs
from dgl.data.utils import save_graphs
save_graphs("/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/DetailedLabel/Models/Pandas_G1_test_DetailedLabel.bin", [G1_test])
save_graphs("/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/DetailedLabel/Models/Pandas_G1_test_ab_DetailedLabel.bin", [G1_test_ab])

np.savetxt('/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/DetailedLabel/Models/Pandas_test_pred1_DetailedLabel.txt', test_pred1, fmt='%d')
np.savetxt('/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/DetailedLabel/Models/Pandas_test_pred1_ab_DetailedLabel.txt', test_pred1_ab, fmt='%d')

### Load graphs and predictions

In [8]:
# loading Graphs and Predictions
from dgl.data.utils import load_graphs
import numpy as np

Test_Graph = load_graphs("/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/DetailedLabel/Models/Pandas_G1_test_DetailedLabel.bin")
Test_Graph = Test_Graph[0][0]
Test_Graph_ab = load_graphs("/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/DetailedLabel/Models/Pandas_G1_test_ab_DetailedLabel.bin")
Test_Graph_ab = Test_Graph_ab[0][0]

Test_pred_Graph = np.loadtxt('/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/DetailedLabel/Models/Pandas_test_pred1_DetailedLabel.txt', dtype=int)
Test_pred_Graph_ab = np.loadtxt('/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/DetailedLabel/Models/Pandas_test_pred1_ab_DetailedLabel.txt', dtype=int)


actual1 = Test_Graph.edata['label']
actual1_ab = Test_Graph_ab.edata['label']


# Test the loading
print(actual1)
print(actual1_ab)
aa = actual1 == actual1_ab
aa = aa.numpy()
print(np.where(aa == False))

print(Test_pred_Graph)
print(Test_pred_Graph_ab)
aa = Test_pred_Graph == Test_pred_Graph_ab

print(type(Test_pred_Graph))
print(type(Test_pred_Graph_ab))
print(type(aa))

print(aa)
print(np.where(aa == False))

print(Test_Graph)
print(Test_Graph_ab)

tensor([1, 1, 0,  ..., 0, 0, 0])
tensor([1, 0, 0,  ..., 0, 0, 0])
(array([     1,      9,     14, ..., 138035, 138036, 138040]),)
[1 1 0 ... 0 0 0]
[1 0 0 ... 0 1 0]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[ True False  True ...  True False  True]
(array([     1,      9,     14, ..., 138036, 138040, 138048]),)
Graph(num_nodes=104640, num_edges=138050,
      ndata_schemes={'feature': Scheme(shape=(1, 76), dtype=torch.float32)}
      edata_schemes={'DetailedLabel': Scheme(shape=(), dtype=torch.int64), 'Edge_indx': Scheme(shape=(), dtype=torch.int64), 'label': Scheme(shape=(), dtype=torch.int64), 'h': Scheme(shape=(1, 76), dtype=torch.float32)})
Graph(num_nodes=276100, num_edges=138050,
      ndata_schemes={'feature': Scheme(shape=(1, 76), dtype=torch.float32)}
      edata_schemes={'DetailedLabel': Scheme(shape=(), dtype=torch.int64), 'Edge_indx': Scheme(shape=(), dtype=torch.int64), 'label': Scheme(shape=(), dtype=torch.int64), 'h': Scheme(shape=(1, 76), d

### Models testing on the graphs

In [9]:
model1_test = Model(76, size_embedding, 76, F.relu, 0.2).cuda()
model1_test.load_state_dict(th.load('/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/DetailedLabel/Models/Pandas_GNN_DetailedLabel.pt'))
model1_test.eval()

model1_test_ab = Model(76, size_embedding, 76, F.relu, 0.2).cuda()
model1_test_ab.load_state_dict(th.load('/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/DetailedLabel/Models/Pandas_GNN_ab_DetailedLabel.pt'))
model1_test_ab.eval()


print("\\\\\\\\\\\\\\\\ NORMAL \\\\\\\\\\\\\\\\\\")

Test_Graph1 = Test_Graph.to('cuda:0')
node_features_test1 = Test_Graph1.ndata['feature']
edge_features_test1 = Test_Graph1.edata['h']

test_pred1 = model1_test(Test_Graph1, node_features_test1, edge_features_test1).cuda()
test_pred1 = test_pred1.argmax(1)
test_pred1 = th.Tensor.cpu(test_pred1).detach().numpy()

print('Metrics : ')
print("Accuracy : ", sklearn.metrics.accuracy_score(actual1, test_pred1))
print("Precision : ", sklearn.metrics.precision_score(actual1, test_pred1, labels = [0,1]))
print("Recall : ", sklearn.metrics.recall_score(actual1, test_pred1, labels = [0,1]))
print("f1_score : ", sklearn.metrics.f1_score(actual1, test_pred1, labels = [0,1]))


print("\\\\\\\\\\\\\\\\ ABLATION \\\\\\\\\\\\\\\\\\")

Test_Graph_ab1 = Test_Graph_ab.to('cuda:0')
node_features_test1_ab = Test_Graph_ab1.ndata['feature']
edge_features_test1_ab = Test_Graph_ab1.edata['h']

test_pred1_ab = model1_test_ab(Test_Graph_ab1, node_features_test1_ab, edge_features_test1_ab).cuda()
test_pred1_ab = test_pred1_ab.argmax(1)
test_pred1_ab = th.Tensor.cpu(test_pred1_ab).detach().numpy()

print('Metrics : ')
print("Accuracy : ", sklearn.metrics.accuracy_score(actual1_ab, test_pred1_ab))
print("Precision : ", sklearn.metrics.precision_score(actual1_ab, test_pred1_ab, labels = [0,1]))
print("Recall : ", sklearn.metrics.recall_score(actual1_ab, test_pred1_ab, labels = [0,1]))
print("f1_score : ", sklearn.metrics.f1_score(actual1_ab, test_pred1_ab, labels = [0,1]))

\\\\\\\\ NORMAL \\\\\\\\\
Metrics : 
Accuracy :  0.9907642158638175
Precision :  0.9673149492017417
Recall :  0.9955486511516745
f1_score :  0.9812287443134137
\\\\\\\\ ABLATION \\\\\\\\\
Metrics : 
Accuracy :  0.9833973198116625
Precision :  0.9463483065647457
Recall :  0.9875123233651002
f1_score :  0.9664922078301804


### Extract the 3% of difference

In [10]:
print(Test_Graph.edata['Edge_indx'])
print(Test_Graph_ab.edata['Edge_indx'])

tensor([     0,  71207,     22,  ..., 138044, 138045, 138046])
tensor([     0,      1,      2,  ..., 138047, 138048, 138049])


In [42]:
actual1_np = actual1.numpy()
aa = actual1_np == Test_pred_Graph
wrong_preds = np.where(aa == False)[0]
print(wrong_preds)

actual1_ab_np = actual1_ab.numpy()
bb = actual1_ab_np == Test_pred_Graph_ab
wrong_preds_ab = np.where(bb == False)[0]
print(wrong_preds_ab)

[  3288   3495   3825 ... 137915 137984 137995]
[   107    165    253 ... 138011 138038 138048]


In [43]:
wpred = Test_Graph.edata['Edge_indx'][wrong_preds]
wpred_ab = Test_Graph_ab.edata['Edge_indx'][wrong_preds_ab]

print(wpred)
print(wpred_ab)

edges_to_explain = set(wpred_ab.numpy()) - set(wpred.numpy())

print(len(edges_to_explain))
print(edges_to_explain)

tensor([ 69962,     76,  87306,  ..., 137736, 137871, 137893])
tensor([   107,    165,    253,  ..., 138011, 138038, 138048])
2867
{65536, 8194, 122884, 49158, 81936, 49170, 49171, 49174, 65560, 98334, 73758, 24609, 98343, 73768, 90152, 131111, 98349, 81968, 73786, 65595, 106556, 98369, 57412, 16453, 8262, 8263, 41030, 81990, 90181, 16459, 49233, 65620, 90201, 41056, 57440, 90209, 32868, 24678, 107, 57451, 41070, 24689, 73842, 106623, 98432, 24705, 106626, 57471, 123014, 41100, 8334, 98456, 106649, 8346, 24736, 114850, 165, 98469, 106662, 98477, 123055, 41138, 131250, 114870, 16569, 106688, 16580, 49348, 65734, 57540, 114891, 65742, 24787, 131287, 49376, 24804, 90341, 106724, 41191, 106725, 106726, 106739, 82166, 41207, 123131, 253, 57597, 16641, 33025, 33027, 123140, 8453, 49415, 98567, 16649, 8459, 123156, 57625, 49434, 8475, 33056, 33063, 82218, 24883, 65843, 8501, 16700, 106816, 57665, 106824, 41289, 41295, 8529, 339, 115030, 41310, 131432, 57707, 82285, 123246, 65904, 131443, 4952

In [44]:
indx_to_explain = []

for x in edges_to_explain:
    indx_to_explain.append((Test_Graph.edata['Edge_indx'] == x).nonzero(as_tuple=True)[0].item())

# print(indx_to_explain)

df_indx = pd.DataFrame(columns = ['Edge_indx', 'label'])

for x in indx_to_explain:
    df_indx.loc[-1] = [x, Test_Graph.edata['label'][x].item()]  # adding a row
    df_indx.index = df_indx.index + 1  # shifting index

    
df_indx = df_indx.sort_values('label')
print(df_indx)

print('nb attacks :', len(df_indx.loc[df_indx['label'] == 1]))
print('nb benign :', len(df_indx.loc[df_indx['label'] == 0]))

      Edge_indx  label
2866      98607      0
1024      39625      0
1023       8457      0
1022     112820      0
1021      66589      0
...         ...    ...
2582      10636      1
2581      61213      1
1748      27567      1
2036      55686      1
1521     106883      1

[2867 rows x 2 columns]
nb attacks : 308
nb benign : 2559


In [69]:
import pandas as pd

df2_indx = pd.DataFrame(columns = ['Edge_indx', 'label', 'in_out_degrees'])

for x in df_indx['label'].unique():
    # select the edges having a neighborhood subgraph with 100 <= nb_edges =< 200
    list1 = df_indx.loc[df_indx['label'] == x]['Edge_indx']
    for y in list1:
        source_node = th.Tensor.cpu(Test_Graph.edges()[0][int(y)]).detach().numpy()
        in_out_degrees = Test_Graph.in_degrees(source_node) + Test_Graph.out_degrees(source_node)
        
        if (100 <= in_out_degrees.item() <= 200) :
            df2_indx.loc[-1] = [int(y), x, in_out_degrees.item()]  # adding a row
            df2_indx.index = df2_indx.index + 1  # shifting index

print(df2_indx)

    Edge_indx  label  in_out_degrees
94       8457      0             120
93       8477      0             120
92       7491      0             155
91       4970      0             110
90       5031      0             127
..        ...    ...             ...
4        7893      1             180
3        7887      1             180
2        7892      1             180
1        7855      1             180
0        7890      1             180

[95 rows x 3 columns]


In [80]:
print(Test_Graph.edata['label'][4970])
print(Test_Graph.edata['label'][7892])

tensor(0)
tensor(1)


In [92]:
df2_indx.to_csv(f'/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/DetailedLabel/Models/Edges_to_explain_DetailedLabel.csv', sep=',', index = False)

### GNN-Edge-Explainer (Local Explanation for each one of the 3% edges)

### Edge feature explanations

In [13]:
# Local Explanation ***********************************************************************
from math import sqrt
from tqdm import tqdm
from dgl import EID, NID, khop_out_subgraph, khop_in_subgraph

import torch.nn as nn
import torch as th
import dgl


# init mask
def init_efeat_masks(graph, efeat):
    # efeat.size() = torch.Size([nb_edges, 1, 76])
    efeat_size = efeat.size()[2]
    num_edges = graph.num_edges()
    num_nodes = graph.num_nodes()
    device = efeat.device
    std = 0.1
    # feat_mask = [[f1, f2, .... fn]] / n = nb_features
    efeat_mask = nn.Parameter(th.randn(1, efeat_size, device=device) * std)
    std = nn.init.calculate_gain("relu") * sqrt(2.0 / (2 * num_edges))
    # edge_mask = [e1, e2, .... em] / m = nb_edges
    # edge_mask = nn.Parameter(th.randn(num_edges, device=device) * std)
    return efeat_mask


# Regularization loss
def loss_regularize_efeat(loss, feat_mask):
    # epsilon for numerical stability
    eps = 1e-15
    # From self GNNExplainer self
    alpha1 = 0.005,
    alpha2 = 1.0
    beta1 = 1.0
    beta2 = 0.1

    #edge_mask = edge_mask.sigmoid()
    # Edge mask sparsity regularization
    #loss = loss + th.from_numpy(alpha1 * th.Tensor.cpu(th.sum(edge_mask)).detach().numpy()).cuda()
    # Edge mask entropy regularization
    #ent = -edge_mask * th.log(edge_mask + eps) - (1 - edge_mask) * th.log(1 - edge_mask + eps)
    #loss = loss + alpha2 * ent.mean()

    feat_mask = feat_mask.sigmoid()
    # Feature mask sparsity regularization
    loss = loss + beta1 * th.mean(feat_mask)
    # Feature mask entropy regularization
    ent = -feat_mask * th.log(feat_mask + eps) - (
        1 - feat_mask
    ) * th.log(1 - feat_mask + eps)
    loss = loss + beta2 * ent.mean()
    return loss



# Edge mask
def explain_edge_features(model, edge_id, graph, node_feat, edge_feat, **kwargs):
    model = model.to(graph.device)
    model.eval()

    # Extract source node-centered k-hop subgraph from the edge_id and its associated node and edge features.
    num_hops = 3
    source_node = th.Tensor.cpu(graph.edges()[0][edge_id]).detach().numpy()
    dest_node = th.Tensor.cpu(graph.edges()[1][edge_id]).detach().numpy()
    edge_h = graph.edata['h'][edge_id]
    sg, inverse_indices = khop_in_subgraph(graph, source_node, num_hops)
    
    # The edge is added in last position
    sg = dgl.add_edges(sg, inverse_indices.item(), dest_node)
    # So we can add its features in last position
    sg.edata['DetailedLabel'][-1] = graph.edata['DetailedLabel'][edge_id]
    sg.edata['Edge_indx'][-1] = graph.edata['Edge_indx'][edge_id]
    sg.edata['label'][-1] = graph.edata['label'][edge_id]
    sg.edata['h'][-1][0] = graph.edata['h'][edge_id]

    # edge_indice using edge_feature
    #for indx, nd_id in enumerate(sg.edges()[0]):
    #    if inverse_indices == nd_id :
    #        if (sg.edata['h'][indx][0] == edge_h[0]).all() :
    #            edge_indice = indx
    #            break
    
    # edge_indice using Edge_indx
    for indx, edge_indx in enumerate(sg.edata["Edge_indx"]):
        if (edge_indx == graph.edata["Edge_indx"][edge_id]).all() :
                edge_indice = indx
                break

    # EID = NID = _ID
    # tensor([0, 1, 2, 4]) : nodes and edges ids
    sg_edges = sg.edata[EID].long()
    sg_nodes = sg.ndata[NID].long()

    #print()
    edge_feat = edge_feat[sg_edges]
    node_feat = node_feat[sg_nodes]
    
    
    # Get the initial prediction.
    #with th.no_grad():
    #    # logits = model(g = sg, nfeats = node_feat, efeats = edge_feat, **kwargs)
    #    logits = model(g = sg, nfeats = node_feat, efeats = edge_feat)
    #    pred_label = logits.argmax(dim=-1)
    
    edge_label_1 = sg.edata["label"][edge_indice]

    # edge_mask
    efeat_mask = init_efeat_masks(sg, edge_feat)
    params = [efeat_mask]
    optimizer = th.optim.Adam(params, lr = 0.01)
    
    from sklearn.utils import class_weight
    # class_weights2 = class_weight.compute_class_weight(class_weight = 'balanced', classes = np.unique(sg.edata['label'].cpu().numpy()), y = sg.edata['label'].cpu().numpy())
    # class_weights2 = class_weight.compute_class_weight(class_weight = 'balanced', classes = np.array([0, 1]), y = sg.edata['label'].cpu().numpy())
    # class_weights2 = th.FloatTensor(class_weights2).cuda()
    # criterion2 = nn.CrossEntropyLoss(weight = class_weights2)
    criterion2 = nn.CrossEntropyLoss()
    train_mask2 = th.ones(len(sg.edata['h']), dtype=th.bool)
    import datetime
    
    #print(f'explanation starts at {datetime.datetime.now()}')
    #print("nb edges : ", sg.num_edges())
    #print("nb nodes : ", sg.num_nodes())
    
    
    for epoch in range(1, 1000):
        optimizer.zero_grad()
        # Apply edge feature mask
        h = edge_feat * efeat_mask.sigmoid()
        # Edge mask
        logits = model(g = sg, nfeats = node_feat, efeats = h).cuda()
        # logits = model(g = sg, nfeats = node_feat, efeats = h)
        # pred_label = tensor([0, 0, 0,  ..., 0, 1, 0], device='cuda:0')
        # logits = tensor([[ 0.0059,  0.0517], [-0.0075,  0.0101], ..., device='cuda:0', grad_fn=<IndexBackward0>)
        # log_probs = logits.log_softmax(dim=-1)
        # loss = -log_probs[edge_indice, pred_label[edge_indice]]
        # loss11 = criterion2(logits[train_mask2], pred_label[train_mask2])
        loss = criterion2(logits[edge_indice], edge_label_1)
        # loss = loss_regularize_efeat(loss11, edge_mask)
        # loss = loss_regularize(loss, efeat_mask, edge_mask)
        
        #if epoch % 100 == 0:
            #print("+++++++++++++++")
            #print(f'epoch number {epoch}, CrossEntropy_Loss = {loss11}, final_loss = {loss}, time = {datetime.datetime.now()}')
            #print("edge_mask : ", edge_mask.detach().sigmoid())
        
        loss.backward()
        optimizer.step()

    efeat_mask = efeat_mask.detach().sigmoid().squeeze()
    return edge_indice, sg, efeat_mask, loss.item()

In [17]:
#Test_Graph1 = Test_Graph.to('cuda:0')
#node_features_test1 = Test_Graph1.ndata['feature']
#edge_features_test1 = Test_Graph1.edata['h']

edge_indice, sub_graph, efeat_mask, loss = explain_edge_features(model1_test, 7855, Test_Graph1, node_features_test1, edge_features_test1)

print("final results : ")
print("efeat_mask : ", efeat_mask)
print("sub_graph : ", sub_graph)
print("edge_indice : ", edge_indice)
print("loss : ", loss)

final results : 
efeat_mask :  tensor([0.7393, 0.1131, 0.3008, 0.9576, 0.7406, 0.7901, 0.0945, 0.1025, 0.4467,
        0.4568, 0.4682, 0.4651, 0.5199, 0.1051, 0.0908, 0.1000, 0.1227, 0.8637,
        0.7001, 0.3191, 0.0909, 0.5284, 0.0451, 0.8932, 0.7835, 0.0952, 0.0629,
        0.1032, 0.8911, 0.0495, 0.0609, 0.0911, 0.7754, 0.7283, 0.9467, 0.8582,
        0.7914, 0.7736, 0.4878, 0.3749, 0.9526, 0.7633, 0.5007, 0.2812, 0.5196,
        0.0838, 0.1844, 0.0367, 0.3369, 0.9572, 0.0719, 0.3592, 0.4096, 0.3712,
        0.0693, 0.2220, 0.0792, 0.4681, 0.5832, 0.1230, 0.7852, 0.7834, 0.9361,
        0.0930, 0.9714, 0.5586, 0.0883, 0.7608, 0.9114, 0.9450, 0.1228, 0.4393,
        0.0703, 0.0927, 0.4615, 0.4929], device='cuda:0')
sub_graph :  Graph(num_nodes=36586, num_edges=2805,
      ndata_schemes={'feature': Scheme(shape=(1, 76), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'DetailedLabel': Scheme(shape=(), dtype=torch.int64), 'Edge_indx': Scheme(shap

### Results (Final_results_DetailedLabel_Final.csv)

In [19]:
import pandas as pd

results_dff = pd.read_csv(f'/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/DetailedLabel/Models/Final_results_DetailedLabel_Final.csv', encoding="ISO-8859–1", dtype = str)
print(results_dff)

results_dff = results_dff.apply(pd.to_numeric)

print(results_dff.dtypes)

   Edge_indx label in_out_degrees                loss
0     8457.0   0.0          120.0   0.660783760547638
1     8477.0   0.0          120.0  0.6234775650501252
2     7491.0   0.0          155.0  1.7774477297067643
3     4970.0   0.0          110.0   1.306601631641388
4     5031.0   0.0          127.0  1.1317549550533295
..       ...   ...            ...                 ...
90    7893.0   1.0          180.0  234.10380819439888
91    7887.0   1.0          180.0   234.0758547180891
92    7892.0   1.0          180.0  234.00118369698524
93    7855.0   1.0          180.0  233.96855336725713
94    7890.0   1.0          180.0   7.371409423321275

[95 rows x 4 columns]
Edge_indx         float64
label             float64
in_out_degrees    float64
loss              float64
dtype: object


In [20]:
min_mutual_info = results_dff.loc[results_dff.groupby('label').loss.idxmin()]
max_mutual_info = results_dff.loc[results_dff.groupby('label').loss.idxmax()]

print('min loss value : ')
print(min_mutual_info)
print()
print('max loss value : ')
print(max_mutual_info)

min loss value : 
    Edge_indx  label  in_out_degrees      loss
24    22530.0    0.0           132.0  0.389771
51     7853.0    1.0           180.0  7.030831

max loss value : 
    Edge_indx  label  in_out_degrees        loss
7      7519.0    0.0           155.0    1.903420
58     7874.0    1.0           180.0  234.208979


### Explain the two selected edges

### Feature Results (Final_results_DetailedLabel_Final.csv)

In [21]:
import pandas as pd

x_testt = pd.read_csv(f'/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/input/Dataset/XAI/X_test4.csv', encoding="ISO-8859–1", dtype = str)
colss = x_testt.columns

cols_to_norm1 = list(set(list(x_testt.iloc[:, :].columns )) - set(list(['label', ' Source IP', ' Destination IP', 'DetailedLabel'])))
print(cols_to_norm1)
print(len(cols_to_norm1))

[' Total Backward Packets', ' Idle Min', ' Subflow Bwd Bytes', ' Fwd Packet Length Min', ' Subflow Bwd Packets', ' Avg Fwd Segment Size', ' Fwd IAT Max', 'Fwd IAT Total', ' Idle Max', 'Total Length of Fwd Packets', 'Bwd Packet Length Max', ' Bwd PSH Flags', 'Bwd Avg Bulk Rate', ' Packet Length Mean', 'Bwd IAT Total', ' Flow IAT Mean', ' Avg Bwd Segment Size', ' Down/Up Ratio', ' Fwd URG Flags', ' Total Length of Bwd Packets', ' URG Flag Count', ' Bwd URG Flags', ' SYN Flag Count', ' Packet Length Variance', ' Active Max', ' Flow IAT Max', ' Bwd IAT Mean', ' Bwd Packet Length Mean', ' Active Std', ' Idle Std', 'Fwd PSH Flags', ' Flow IAT Std', ' Fwd Packet Length Mean', ' Flow IAT Min', ' PSH Flag Count', ' ACK Flag Count', ' act_data_pkt_fwd', ' Init_Win_bytes_backward', ' Bwd Avg Packets/Bulk', 'Subflow Fwd Packets', ' Min Packet Length', ' Bwd Packets/s', ' Fwd Avg Packets/Bulk', ' Fwd Packet Length Std', ' Fwd Avg Bulk Rate', ' Fwd IAT Min', ' Packet Length Std', 'Init_Win_bytes_for

#### Attack edge features

In [19]:
ef_edge_indice_1, ef_sub_graph_1, efeat_mask_1, ef_loss_1 = explain_edge_features(model1_test, 7855, Test_Graph1, node_features_test1, edge_features_test1)

print("final results : ")
print("edge explained : ", 7855)
print("efeat_mask : ", efeat_mask_1)
print("sub_graph : ", ef_sub_graph_1)
print("edge_indice : ", ef_edge_indice_1)
print("loss : ", ef_loss_1)

final results : 
edge explained :  7855
efeat_mask :  tensor([0.7384, 0.1171, 0.3134, 0.9632, 0.7514, 0.7976, 0.0794, 0.0925, 0.4483,
        0.4441, 0.4421, 0.5424, 0.5137, 0.1006, 0.0956, 0.1093, 0.1474, 0.8502,
        0.7578, 0.2739, 0.0895, 0.5556, 0.0487, 0.8784, 0.7746, 0.0941, 0.0603,
        0.1171, 0.9048, 0.0485, 0.0583, 0.0948, 0.8063, 0.7377, 0.9445, 0.8701,
        0.8101, 0.8189, 0.5504, 0.3423, 0.9532, 0.7831, 0.5009, 0.2832, 0.4967,
        0.0931, 0.1952, 0.0387, 0.3278, 0.9602, 0.0697, 0.4087, 0.4827, 0.3617,
        0.0681, 0.2120, 0.0773, 0.4601, 0.5566, 0.1307, 0.7796, 0.7941, 0.9358,
        0.0986, 0.9698, 0.4915, 0.0793, 0.7207, 0.9099, 0.9543, 0.1134, 0.3557,
        0.0713, 0.0939, 0.5113, 0.4833], device='cuda:0')
sub_graph :  Graph(num_nodes=36586, num_edges=2805,
      ndata_schemes={'feature': Scheme(shape=(1, 76), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'DetailedLabel': Scheme(shape=(), dtype=torch.int64), '

In [20]:
print(Test_Graph1.edata['DetailedLabel'][7855])

tensor(9, device='cuda:0')


In [26]:
feature_weights_final = pd.DataFrame(columns = ['Feature_name', 'Importance_weight'])

for indx, x in enumerate(cols_to_norm1):
    feature_weights_final.loc[-1] = [x, efeat_mask_1.cpu().detach()[indx].item()]  # adding a row
    feature_weights_final.index = feature_weights_final.index + 1  # shifting index

print(feature_weights_final)

               Feature_name  Importance_weight
75   Total Backward Packets           0.738429
74                 Idle Min           0.117057
73        Subflow Bwd Bytes           0.313384
72    Fwd Packet Length Min           0.963157
71      Subflow Bwd Packets           0.751413
..                      ...                ...
4       Fwd Header Length.1           0.355730
3      min_seg_size_forward           0.071259
2               Fwd IAT Std           0.093869
1        Fwd Avg Bytes/Bulk           0.511335
0        Bwd Avg Bytes/Bulk           0.483261

[76 rows x 2 columns]


In [27]:
feature_weights_final = feature_weights_final.sort_values(by = 'Importance_weight', ascending = False)
print(feature_weights_final)

              Feature_name  Importance_weight
11                Protocol           0.969751
72   Fwd Packet Length Min           0.963157
26       Max Packet Length           0.960226
6    Bwd Packet Length Min           0.954250
35       Min Packet Length           0.953229
..                     ...                ...
49            Bwd IAT Mean           0.060296
45           Fwd PSH Flags           0.058340
53          SYN Flag Count           0.048655
46                Idle Std           0.048487
28  Init_Win_bytes_forward           0.038663

[76 rows x 2 columns]


In [28]:
print(feature_weights_final.iloc[0:11])

               Feature_name  Importance_weight
11                 Protocol           0.969751
72    Fwd Packet Length Min           0.963157
26        Max Packet Length           0.960226
6     Bwd Packet Length Min           0.954250
35        Min Packet Length           0.953229
41           PSH Flag Count           0.944505
13           ECE Flag Count           0.935783
7            RST Flag Count           0.909857
47               Active Std           0.904808
52   Packet Length Variance           0.878375
40           ACK Flag Count           0.870101


In [30]:
feature_weights_final.style

,Feature_name,Importance_weight
11,Protocol,0.969751
72,Fwd Packet Length Min,0.963157
26,Max Packet Length,0.960226
6,Bwd Packet Length Min,0.954250
35,Min Packet Length,0.953229
41,PSH Flag Count,0.944505
13,ECE Flag Count,0.935783
7,RST Flag Count,0.909857
47,Active Std,0.904808
52,Packet Length Variance,0.878375


In [31]:
feature_weights_final.to_csv(f'/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/DetailedLabel/feature_weights_final.csv', sep=',', index = False)

#### Benign edge features

In [55]:
Test_Graph1 = Test_Graph.to('cuda:0')
node_features_test1 = Test_Graph1.ndata['feature']
edge_features_test1 = Test_Graph1.edata['h']

ef_edge_indice_0, ef_sub_graph_0, efeat_mask_0, ef_loss_0 = explain_edge_features(model1_test, 22530, Test_Graph1, node_features_test1, edge_features_test1)

print("final results : ")
print("edge explained : ", 22530)
print("efeat_mask : ", efeat_mask_0)
print("sub_graph : ", ef_sub_graph_0)
print("edge_indice : ", ef_edge_indice_0)
print("loss : ", ef_loss_0)

final results : 
edge explained :  22530
efeat_mask :  tensor([0.4816, 0.5007, 0.5767, 0.4624, 0.5317, 0.5124, 0.4637, 0.5070, 0.4908,
        0.4866, 0.5307, 0.5152, 0.4660, 0.4570, 0.4952, 0.4999, 0.5416, 0.4782,
        0.4473, 0.5205, 0.5140, 0.5164, 0.5035, 0.5348, 0.4845, 0.4859, 0.5197,
        0.4830, 0.5335, 0.5089, 0.5073, 0.4968, 0.4962, 0.4495, 0.4737, 0.5159,
        0.4977, 0.5107, 0.4599, 0.5157, 0.4920, 0.4773, 0.5218, 0.4812, 0.5048,
        0.4795, 0.5145, 0.4613, 0.4425, 0.5076, 0.4758, 0.5187, 0.5253, 0.5489,
        0.4970, 0.4762, 0.4831, 0.4581, 0.5355, 0.4910, 0.4610, 0.4704, 0.5118,
        0.4939, 0.4662, 0.5314, 0.4949, 0.4747, 0.4962, 0.5629, 0.5096, 0.5165,
        0.4841, 0.4758, 0.4823, 0.5317], device='cuda:0')
sub_graph :  Graph(num_nodes=54770, num_edges=87,
      ndata_schemes={'feature': Scheme(shape=(1, 76), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'DetailedLabel': Scheme(shape=(), dtype=torch.int64), 'E

In [56]:
feature_weights_final_benign = pd.DataFrame(columns = ['Feature_name', 'Importance_weight'])

for indx, x in enumerate(cols_to_norm1):
    feature_weights_final_benign.loc[-1] = [x, efeat_mask_0.cpu().detach()[indx].item()]  # adding a row
    feature_weights_final_benign.index = feature_weights_final_benign.index + 1  # shifting index

print(feature_weights_final_benign)

               Feature_name  Importance_weight
75   Total Backward Packets           0.481578
74                 Idle Min           0.500717
73        Subflow Bwd Bytes           0.576703
72    Fwd Packet Length Min           0.462423
71      Subflow Bwd Packets           0.531668
..                      ...                ...
4       Fwd Header Length.1           0.516547
3      min_seg_size_forward           0.484059
2               Fwd IAT Std           0.475823
1        Fwd Avg Bytes/Bulk           0.482264
0        Bwd Avg Bytes/Bulk           0.531683

[76 rows x 2 columns]


In [57]:
feature_weights_final_benign = feature_weights_final_benign.sort_values(by = 'Importance_weight', ascending = False)
print(feature_weights_final_benign.iloc[0:11])

               Feature_name  Importance_weight
73        Subflow Bwd Bytes           0.576703
6     Bwd Packet Length Min           0.562913
22                Idle Mean           0.548867
59     Avg Bwd Segment Size           0.541600
17    Fwd Packet Length Max           0.535543
52   Packet Length Variance           0.534765
47               Active Std           0.533494
0        Bwd Avg Bytes/Bulk           0.531683
71      Subflow Bwd Packets           0.531668
10      Average Packet Size           0.531412
65    Bwd Packet Length Max           0.530688


In [58]:
feature_weights_final_benign.style

,Feature_name,Importance_weight
73,Subflow Bwd Bytes,0.576703
6,Bwd Packet Length Min,0.562913
22,Idle Mean,0.548867
59,Avg Bwd Segment Size,0.541600
17,Fwd Packet Length Max,0.535543
52,Packet Length Variance,0.534765
47,Active Std,0.533494
0,Bwd Avg Bytes/Bulk,0.531683
71,Subflow Bwd Packets,0.531668
10,Average Packet Size,0.531412


In [59]:
feature_weights_final_benign.to_csv(f'/home/ahmed/GNN-Based-ANIDS/GNN-Based-ANIDS/jupyter_notebooks/XAI/GNNExplainer/DetailedLabel/feature_weights_final_benign.csv', sep=',', index = False)